In [2]:
import pandas as pd

# Cargar el dataset
data = pd.read_csv('amazon.tsv', sep='\t', nrows=50_000)


In [38]:
# Seleccionar solo las columnas necesarias
data = data[['customer_id', 'product_id', 'star_rating']]

# Filtrar las valoraciones nulas si las hay
data = data.dropna()
data.sample(1)

,customer_id,product_id,star_rating
44396,34682965,0471738948,5


In [4]:
# Crear la matriz de usuario-producto
user_product_matrix = data.pivot_table(index='customer_id', columns='product_id', values='star_rating').fillna(0)


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud de coseno entre los usuarios
user_similarity = cosine_similarity(user_product_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_product_matrix.index, columns=user_product_matrix.index)


In [39]:
def recommend_products(user_id, num_recommendations=5):
    # Obtener la fila de similitudes para el usuario dado
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]
    
    # Obtener los productos que han sido calificados por usuarios similares
    similar_users_ratings = user_product_matrix.loc[similar_users.index]
    
    # Calcular el promedio ponderado de las valoraciones de productos por usuarios similares
    mean_ratings = similar_users_ratings.T.dot(similar_users) / similar_users.sum()
    
    # Filtrar productos que el usuario ya ha calificado
    user_rated_products = user_product_matrix.loc[user_id][user_product_matrix.loc[user_id] > 0].index
    recommendations = mean_ratings.drop(user_rated_products).sort_values(ascending=False)
    
    return recommendations.head(num_recommendations)

# Ejemplo de uso
recommended_products = recommend_products(34682965, 5)
recommended_products_array = recommended_products.index.tolist()
# Leer el dataset
df = pd.read_csv('amazon.tsv', sep='\t', nrows=50_000)

# Filtrar el DataFrame para obtener solo las filas con los product_id recomendados
filtered_data = df[df['product_id'].isin(recommended_products_array)]

# Eliminar duplicados basados en el product_id
unique_recommended_titles = filtered_data.drop_duplicates(subset='product_id')[['product_id', 'product_title']]

# Imprimir los títulos de los productos recomendados
print(unique_recommended_titles)

       product_id                                      product_title
1832   158322713X                            A Man Without a Country
3296   1932226397  Call to Action: Secret Formulas to Improve Onl...
6002   1885167628  Bag the Elephant!: How to Win and Keep Big Cus...
23812  1560257679  Fog Facts : Searching for Truth in the Land of...
29251  0471730440       Persuasion: The Art of Getting What You Want
